# FINAL PROJECT : 
###      DengAI: Predicting Disease Spread		
####               HOSTED BY DRIVENDATA

- ###### Asjad ALi
- ###### 18101144


#########################################################################################################################

### Reading the File
- ###### df as Training Data
- ###### df2 as Testing Data

In [22]:
import pandas as pd
df=pd.read_csv(r'C:\Users\Peritus02\Desktop\Project\dengue_features_train.csv')
df2=pd.read_csv(r'C:\Users\Peritus02\Desktop\Project\dengue_features_test.csv')

# Pre-Processing

###### Checking if the Training Data Frame has the Missing Value Columns

In [23]:
df.columns [df.isnull().any()]

Index(['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm'],
      dtype='object')

###### Calculating How much % of the Value are Missing in each Columns to drop the highest Missing Column

In [24]:
missing_percent_training = df.isnull().sum() * 100 / len(df)
missing_percent_training

city                                      0.000000
year                                      0.000000
weekofyear                                0.000000
week_start_date                           0.000000
ndvi_ne                                  13.324176
ndvi_nw                                   3.571429
ndvi_se                                   1.510989
ndvi_sw                                   1.510989
precipitation_amt_mm                      0.892857
reanalysis_air_temp_k                     0.686813
reanalysis_avg_temp_k                     0.686813
reanalysis_dew_point_temp_k               0.686813
reanalysis_max_air_temp_k                 0.686813
reanalysis_min_air_temp_k                 0.686813
reanalysis_precip_amt_kg_per_m2           0.686813
reanalysis_relative_humidity_percent      0.686813
reanalysis_sat_precip_amt_mm              0.892857
reanalysis_specific_humidity_g_per_kg     0.686813
reanalysis_tdtr_k                         0.686813
station_avg_temp_c             

###### Drop ndvi_ne Columns because 10% value was missing and Drop other Two Columns because the do not had any affect of chance on data and are not Changed in whole data set i.e. that are Constant,So removed to keep the model simple

In [25]:
df = df.drop('ndvi_ne' , axis = 1)
df = df.drop('year' , axis = 1)
df = df.drop('week_start_date' , axis = 1)

###### Same Steps for the Testing Data

In [26]:
missing_percent_testing = df2.isnull().sum() * 100 / len(df2)
df2 = df2.drop('ndvi_ne' , axis = 1)
df2 = df2.drop('year' , axis = 1)
df2 = df2.drop('week_start_date' , axis = 1)



###### Converting the Catagorical Features into Numberical 

In [27]:
df["city"] = df["city"].astype('category')
df["city"] = df["city"].cat.codes

df2["city"] = df2["city"].astype('category')
df2["city"] = df2["city"].cat.codes

###### Filling the Remianing Missing Values by mean

In [28]:
training_df= df.iloc[:,0:22]
testing_df = df2.iloc[:,0:22]
training_df=training_df.fillna(training_df.mean())
training_df.columns[training_df.isnull().any()]

testing_df=testing_df.fillna(testing_df.mean())
testing_df.columns[testing_df.isnull().any()]

Index([], dtype='object')

###### Spliting the Training Dataframe into the Features and Labels

In [29]:
training_df_features = training_df.iloc[:, 0:21]
training_df_labels =  training_df.iloc[:, 21]


###### Scaling was Done to Normalize the Data Set so that Regressor might Work Effectivley

In [30]:
from sklearn import preprocessing
training_df_features = preprocessing.scale(training_df_features)
testing_df = preprocessing.scale(testing_df)
testing_df

C:\Users\Peritus02\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int8, int64, float64 were all converted to float64 by the scale function.
  
C:\Users\Peritus02\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int8, int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 7.74596669e-01, -5.64155509e-01, -1.04547788e+00, ...,
         4.00474022e-01, -3.90850783e-01,  1.18939835e+00],
       [ 7.74596669e-01, -4.97311678e-01, -9.98837768e-01, ...,
        -1.32621387e+00, -9.85386391e-02,  6.22325474e-04],
       [ 7.74596669e-01, -4.30467847e-01,  0.00000000e+00, ...,
         1.38854643e-01,  2.52235933e-01, -9.09125583e-01],
       ...,
       [-1.29099445e+00, -1.63092521e-01,  8.91368379e-01, ...,
         3.42068921e-02, -4.49313212e-01,  1.71257606e+00],
       [-1.29099445e+00, -9.62486899e-02,  1.26213145e+00, ...,
        -1.75088611e-01, -3.32388354e-01, -5.19076020e-03],
       [-1.29099445e+00, -2.94048585e-02,  8.12746062e-01, ...,
         3.42068921e-02, -2.15463497e-01, -5.63246985e-01]])

###### Feature Selection is done using  Linear Support Vector Classification to Pick K BEST Features and to reduce Dimenstion

In [31]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(training_df_features, training_df_labels)
model = SelectFromModel(lsvc, prefit=True)
training_df_features = model.transform(training_df_features)
testing_df = model.transform(testing_df)
training_df_features.shape


(1456, 6)

# Model Implementation

###### Keras: Regression-based neural networks

In [32]:
from keras import Sequential
from keras.layers import Dense
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=12, input_dim=6))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','accuracy'])
    return regressor

Using TensorFlow backend.


In [33]:
from keras.wrappers.scikit_learn import KerasRegressor
regressor = KerasRegressor(build_fn=build_regressor, batch_size=32,epochs=100)

###### Fitting the Model on to the Traning Data Set 

In [34]:
regressor.fit(training_df_features,training_df_labels)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1456/1456 [==============================] - 0s 172us/step - loss: 2497.4387 - mean_absolute_error: 24.6322 - acc: 0.0275
Epoch 2/100
1456/1456 [==============================] - 0s 33us/step - loss: 2467.9145 - mean_absolute_error: 24.4184 - acc: 0.0247
Epoch 3/100
1456/1456 [==============================] - 0s 30us/step - loss: 2435.8404 - mean_absolute_error: 24.2081 - acc: 0.0206
Epoch 4/100
1456/1456 [==============================] - 0s 33us/step - loss: 2398.3412 - mean_absolute_error: 23.9664 - acc: 0.0213
Epoch 5/100
1456/1456 [==============================] - 0s 36us/step - loss: 2355.7015 - mean_absolute_error: 23.7482 - acc: 0.0206
Epoch 6/100
1456/1456 [==============================] - 0s 33us/step - loss: 2309.1939 - mean_absolute_error: 23.5076 - acc: 0.0227
Epoch 7/100
1456/1456 [==============================] - 0s 41us/step - loss: 226

Epoch 58/100
1456/1456 [==============================] - 0s 33us/step - loss: 1682.8852 - mean_absolute_error: 20.6249 - acc: 0.0254
Epoch 59/100
1456/1456 [==============================] - 0s 41us/step - loss: 1682.7320 - mean_absolute_error: 20.6421 - acc: 0.0254
Epoch 60/100
1456/1456 [==============================] - 0s 38us/step - loss: 1683.0307 - mean_absolute_error: 20.6967 - acc: 0.0240
Epoch 61/100
1456/1456 [==============================] - 0s 41us/step - loss: 1683.1431 - mean_absolute_error: 20.7015 - acc: 0.0261
Epoch 62/100
1456/1456 [==============================] - 0s 36us/step - loss: 1682.9825 - mean_absolute_error: 20.6309 - acc: 0.0254
Epoch 63/100
1456/1456 [==============================] - 0s 41us/step - loss: 1682.8154 - mean_absolute_error: 20.5853 - acc: 0.0268
Epoch 64/100
1456/1456 [==============================] - 0s 36us/step - loss: 1682.4447 - mean_absolute_error: 20.6414 - acc: 0.0268
Epoch 65/100
1456/1456 [==============================] - 0s 3

###### Prediction Using Model  

In [35]:
y_pred= regressor.predict(testing_df)

###### Writing the Pridiction results into the Submission File as per as Data Driven

In [36]:
out_df=pd.read_csv(r'C:\Users\Peritus02\Desktop\Project\submission_format.csv')
y_pred[y_pred<0] = 0
y_pred=y_pred.astype('int64')
out_df.total_cases = y_pred
out_df.to_csv(r'C:\Users\Peritus02\Desktop\Project\submission_format_new__8.csv')